In [1]:
%load_ext lab_black

In [2]:
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
from pmdarima.model_selection import train_test_split
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib

path_data = here("./data")
os.chdir(path_data)
data_iu_analysis = pd.read_parquet("data_iu_analysis.parquet")

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/backends.py:181: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/backends.py:181: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/backends.py:181: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [3]:
test_size = round(data_iu_analysis.shape[0] * 0.2)

In [4]:
train, test = train_test_split(data_iu_analysis, test_size=test_size)

In [5]:
x_train = train.drop(columns="tournament_make")
x_test = test.drop(columns="tournament_make")
y_train = train["tournament_make"]
y_test = test["tournament_make"]

In [3]:
%pip install xgboost==1.7.3

     |████████████████████████████████| 193.6 MB 45 kB/s /s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.5
    Uninstalling xgboost-1.7.5:
      Successfully uninstalled xgboost-1.7.5
Note: you may need to restart the kernel to use updated packages.


In [6]:
print(xgb.__version__)

1.7.3


In [8]:
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def objective(trial):
    # Define the hyperparameters to search over
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_discrete_uniform("subsample", 0.5, 0.9, 0.1),
        "colsample_bytree": trial.suggest_discrete_uniform(
            "colsample_bytree", 0.5, 0.9, 0.1
        ),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "objective": "binary:logistic",
        "random_state": 42,
    }

    # Train the XGBoost model with the given hyperparameters
    model = xgb.XGBClassifier(**params)
    model.fit(x_train, y_train)

    # Evaluate the model on the test set and return the accuracy as the objective value
    preds = model.predict(x_train)
    accuracy = accuracy_score(y_train, preds)
    return accuracy


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found by Optuna
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters on the full dataset
best_model = xgb.XGBClassifier(**study.best_params)
best_model.fit(x_train, y_train)

[I 2023-04-14 20:14:36,607] A new study created in memory with name: no-name-c66de23c-0261-4227-b972-43296c3d1c7b
/tmp/ipykernel_12923/910578147.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1),
/tmp/ipykernel_12923/910578147.py:14: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
/tmp/ipykernel_12923/910578147.py:15: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~opt

Best hyperparameters: {'learning_rate': 0.400175057747231, 'max_depth': 7, 'subsample': 0.9, 'colsample_bytree': 0.6, 'n_estimators': 50}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.400175057747231,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [10]:
path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(best_model, "model_iu_bball.jlib")

['model_iu_bball.jlib']

In [29]:
prob = best_model.predict_proba(x_test)[:, 1]

In [30]:
prob

array([0.05292255, 0.29561523, 0.13738553, 0.20107803, 0.7674922 ,
       0.23381503, 0.3680813 , 0.37073496, 0.42601648, 0.0309618 ,
       0.960868  , 0.9947214 , 0.0435374 , 0.16509107, 0.01440901],
      dtype=float32)

In [31]:
from sklearn.metrics import f1_score


def find_best_threshold(y_true, y_pred_prob):
    """
    Find the best threshold to use for converting predicted probabilities to binary predictions,
    using the F1 score as the metric to optimize for.

    :param y_true: true labels, array-like of shape (n_samples,)
    :param y_pred_prob: predicted probabilities, array-like of shape (n_samples,)
    :return: best threshold value
    """
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

In [32]:
threshold = find_best_threshold(y_true=y_test, y_pred_prob=prob)
threshold

0.8

In [33]:
prob_threshold = np.where(prob > threshold, 1, 0)

In [34]:
from sklearn.metrics import balanced_accuracy_score, f1_score

In [35]:
balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=prob_threshold)
balanced_accuracy

0.8333333333333333

In [36]:
f1_accuracy = f1_score(y_true=y_test, y_pred=prob_threshold)
f1_accuracy

0.8